In [67]:
# Source: @Jac_McQ
import pathlib
import re
import io
from pathlib import Path
from typing import Any, Union, List
from nbformat import current
test_sentence = "How many words have I written in this blog?"

print(f"{test_sentence} --> {len(test_sentence.split())}")

How many words have I written in this blog? --> 9


In [68]:
# Source: https://github.com/gandreadis/markdown-word-count
def count_words_in_markdown(file_path: str):
    with open(file_path, 'r', encoding='utf8') as f:
        text = f.read()
    
    # Comments    
    text = re.sub(r'<!--(.*?)-->', '', text, flags=re.MULTILINE)
    # Tabs to spaces
    text = text.replace('\t', '    ')
    # More than 1 space to 4 spaces
    text = re.sub(r'[ ]{2,}', '    ', text)
    # Footnotes
    text = re.sub(r'^\[[^]]*\][^(].*', '', text, flags=re.MULTILINE)
    # Indented blocks of code
    text = re.sub(r'^( {4,}[^-*]).*', '', text, flags=re.MULTILINE)
    # Replace newlines with spaces for uniform handling
    text = text.replace('\n', ' ')
    # Custom header IDs
    text = re.sub(r'{#.*}', '', text)
    # Remove images
    text = re.sub(r'!\[[^\]]*\]\([^)]*\)', '', text)
    # Remove HTML tags
    text = re.sub(r'</?[^>]*>', '', text)
    # Remove special characters
    text = re.sub(r'[#*`~\-–^=<>+|/:]', '', text)
    # Remove footnote references
    text = re.sub(r'\[[0-9]*\]', '', text)
    # Remove enumerations
    text = re.sub(r'[0-9#]*\.', '', text)
    
    return len(text.split())

In [69]:
"""
glob pattern elements;
- `**` - Feature known as globstar. Matches all files and zero or more directories and subdirectories. 
- `*` - Matches any string.
- `?` - Matches single character.
- `.md` - Matches files with a `.md` suffix.
"""

abs_path: Union[Path, Any] = pathlib.Path("/Users/wojtek/PycharmProjects/wojtekzubera.dev/content")

all_md: List[Any] = []

for file in abs_path.glob('**/blog/**/*.md'):
    if file.suffix == '.md':
        all_md.append(file)
        
print(f"number of .md files within 'content/blog/' dir:    {len(all_md)}")

total_word_count_md: int = 0

for file in all_md:
    total_word_count_md += count_words_in_markdown(file)
    
print(f"{total_word_count_md} --> words across all the .md files.")

number of .md files within 'content/blog/' dir:    1
21 --> words across all the .md files.


In [70]:
def count_words_in_ipynb(file_path: str, return_type: str = 'markdown'):
    """
    1. Read the notebook as a json files.
    2. Loop through all the cells within the notebook.
    3. If the cell type matches the type we want, count the words using a similar method.
    4. Sum all the counts to get the total for the notebook.
    """
    with io.open(file_path, 'r', encoding='utf8') as f:
        nbk = current.read(f, 'json')
        
    word_count_md: int = 0
    word_count_heading: int = 0
    word_count_code: int = 0
    
    for cell in nbk.worksheets[0].cells:
        if cell.cell_type == 'markdown':
            word_count_md += len(cell['source'].replace('#', '').lstrip().split(' '))
        elif cell.cell_type == 'heading':
            word_count_heading += len(cell['source'].replace('#', '').lstrip().split(' '))
        elif cell.cell_type == 'code':
            word_count_code += len(cell['input'].replace('#', '').lstrip().split(' '))
            
    if return_type == 'markdown':
        return word_count_md
    elif return_type == 'heading':
        return word_count_heading
    elif return_type == 'code':
        return word_count_code
    else:
        return Exception

In [71]:
all_ipynb: List[Any] = []

for file in abs_path.glob('**/blog/**/*.ipynb'):
    if 'checkpoint' not in file.name:
        all_ipynb.append(file)
        
print(f"number of .ipynb files within 'content/blog/' dir:    {len(all_ipynb)}")

number of .ipynb files within 'content/blog/' dir:    1


In [72]:
total_code_count_ipynb: int = 0

for file in all_ipynb:
    total_code_count_ipynb += count_words_in_ipynb(file, 'code')
    
print(f"How many words live in code within the Jupyter notebooks:    {total_code_count_ipynb}")

How many words live in code within the Jupyter notebooks:    727
